![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

#Zero-Shot Clinical Relation Extraction Model

### Zero-shot Relation Extraction to extract relations between clinical entities with no training dataset
This release includes a zero-shot relation extraction model that leverages `BertForSequenceClassificaiton` to return, based on a predefined set of relation candidates (including no-relation / O), which one has the higher probability to be linking two entities.

The dataset will be a csv which contains the following columns: `sentence`, `chunk1`, `firstCharEnt1`, `lastCharEnt1`, `label1`, `chunk2`, `firstCharEnt2`, `lastCharEnt2`, `label2`, `rel`.

The relation types (TeRP, TrAP, PIP, TrNAP, etc...) are described [here](https://www.i2b2.org/NLP/Relations/assets/Relation%20Annotation%20Guideline.pdf)

Let's take a look at the first sentence!

`She states this light-headedness is often associated with shortness of breath and diaphoresis occasionally with nausea`

As we see in the table, the sentences includes a `PIP` relationship (`Medical problem indicates medical problem`), meaning that in that sentence, chunk1 (`light-headedness`) *indicates* chunk2 (`diaphoresis`).

We set a list of candidates tags (`[PIP, TrAP, TrNAP, TrWP, O]`) and candidate sentences (`[light-headedness caused diaphoresis, light-headedness was administered for diaphoresis, light-headedness was not given for diaphoresis, light-headedness worsened diaphoresis]`), meaning that:

- `PIP` is expressed by `light-headedness caused diaphoresis`
- `TrAP` is expressed by `light-headedness was administered for diaphoresis`
- `TrNAP` is expressed by `light-headedness was not given for diaphoresis`
- `TrWP` is expressed by `light-headedness worsened diaphoresis`
- or something generic, like `O` is expressed by `light-headedness and diaphoresis`...

We will get that the biggest probability of is `PIP`, since it's phrase `light-headedness caused diaphoresis` is the most similar relationship expressing the meaning in the original sentence (`light-headnedness is often associated with ... and diaphoresis`)

## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

In [0]:
from johnsnowlabs import nlp, medical

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

import os
import json
import string
import numpy as np
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

spark

## Zero Shot Relation Extraction
Using the pretrained `re_zeroshot_biobert` model, available in Models Hub under the Relation Extraction category.

In [0]:
# Clinical NER

documenter = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentencer = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("tokens")

words_embedder = nlp.WordEmbeddingsModel() \
    .pretrained("embeddings_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"]) \
    .setOutputCol("embeddings")

ner_clinical = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens", "embeddings"]) \
    .setOutputCol("ner_clinical")

ner_clinical_converter = medical.NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_clinical"]) \
    .setOutputCol("ner_clinical_chunks")\
    .setWhiteList(["PROBLEM", "TEST"])      # PROBLEM-TEST-TREATMENT

ner_posology = medical.NerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens", "embeddings"]) \
    .setOutputCol("ner_posology")           

ner_posology_converter = medical.NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_posology"]) \
    .setOutputCol("ner_posology_chunks")\
    .setWhiteList(["DRUG"])                # DRUG-FREQUENCY-DOSAGE-DURATION-FORM-ROUTE-STRENGTH

chunk_merger = medical.ChunkMergeApproach()\
    .setInputCols("ner_clinical_chunks", "ner_posology_chunks")\
    .setOutputCol('merged_ner_chunks')


## ZERO-SHOT RE Starting...

pos_tagger = nlp.PerceptronModel() \
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"]) \
    .setOutputCol("pos_tags")

dependency_parser = nlp.DependencyParserModel() \
    .pretrained("dependency_conllu", "en") \
    .setInputCols(["document", "pos_tags", "tokens"]) \
    .setOutputCol("dependencies")

re_ner_chunk_filter = medical.RENerChunksFilter() \
    .setRelationPairs(["problem-test","problem-drug"]) \
    .setMaxSyntacticDistance(4)\
    .setDocLevelRelations(False)\
    .setInputCols(["merged_ner_chunks", "dependencies"]) \
    .setOutputCol("re_ner_chunks")

re_model = medical.ZeroShotRelationExtractionModel.pretrained("re_zeroshot_biobert", "en", "clinical/models")\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")\
    .setRelationalCategories({
                            "ADE": ["{DRUG} causes {PROBLEM}."],
                            "IMPROVE": ["{DRUG} improves {PROBLEM}.", "{DRUG} cures {PROBLEM}."],
                            "REVEAL": ["{TEST} reveals {PROBLEM}."]})\
    .setMultiLabel(True)

pipeline = nlp.Pipeline() \
    .setStages([documenter,  
                sentencer,
                tokenizer, 
                words_embedder, 
                ner_clinical, 
                ner_clinical_converter,
                ner_posology, 
                ner_posology_converter,
                chunk_merger,
                pos_tagger, 
                dependency_parser, 
                re_ner_chunk_filter, 
                re_model])

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
re_zeroshot_biobert download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


In [0]:
re_model.getClasses()

['IMPROVE', 'ADE', 'REVEAL']

In [0]:
# create Spark DF

sample_text = "Paracetamol can alleviate headache or sickness. An MRI test can be used to find cancer."

data = spark.createDataFrame([[sample_text]]).toDF("text")
data.show(truncate=False)

+---------------------------------------------------------------------------------------+
|text                                                                                   |
+---------------------------------------------------------------------------------------+
|Paracetamol can alleviate headache or sickness. An MRI test can be used to find cancer.|
+---------------------------------------------------------------------------------------+



**Fit the model and transform it with the dataframe.**

In [0]:
model = pipeline.fit(data)
results = model.transform(data)

In [0]:
# relations output

results.selectExpr("explode(relations) as relation").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|relation                                                                                                                                                                                                                                                                                                                                                         |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql import functions as F

results.select(
    F.explode(
        F.arrays_zip(
            results.relations.metadata.alias("metadata"),
            results.relations.result.alias("result")
        )
    ).alias("cols")
) \
.select(
    F.col("cols.metadata").getItem("sentence").alias("sentence"),
    F.col("cols.metadata").getItem("entity1_begin").alias("entity1_begin"),
    F.col("cols.metadata").getItem("entity1_end").alias("entity1_end"),
    F.col("cols.metadata").getItem("chunk1").alias("chunk1"),
    F.col("cols.metadata").getItem("entity1").alias("entity1"),
    F.col("cols.metadata").getItem("entity2_begin").alias("entity2_begin"),
    F.col("cols.metadata").getItem("entity2_end").alias("entity2_end"),
    F.col("cols.metadata").getItem("chunk2").alias("chunk2"),
    F.col("cols.metadata").getItem("entity2").alias("entity2"),
    F.col("cols.metadata").getItem("hypothesis").alias("hypothesis"),
    F.col("cols.metadata").getItem("nli_prediction").alias("nli_prediction"),
    F.col("cols.result").alias("relation"),
    F.col("cols.metadata").getItem("nli_confidence").alias("nli_confidence")  
.show(truncate=70)


+--------+-------------+-----------+-----------+-------+-------------+-----------+--------+-------+------------------------------+--------------+--------+--------------+
|sentence|entity1_begin|entity1_end|     chunk1|entity1|entity2_begin|entity2_end|  chunk2|entity2|                    hypothesis|nli_prediction|relation|nli_confidence|
+--------+-------------+-----------+-----------+-------+-------------+-----------+--------+-------+------------------------------+--------------+--------+--------------+
|       0|            0|         10|Paracetamol|   DRUG|           38|         45|sickness|PROBLEM|Paracetamol improves sickness.|        entail| IMPROVE|          NULL|
|       0|            0|         10|Paracetamol|   DRUG|           26|         33|headache|PROBLEM|Paracetamol improves headache.|        entail| IMPROVE|          NULL|
|       1|           48|         58|An MRI test|   TEST|           80|         85|  cancer|PROBLEM|   An MRI test reveals cancer.|        entail|  REV

## LightPipeline

LightPipelines are Spark NLP specific Pipelines, equivalent to Spark ML Pipeline, but meant to deal with smaller amounts of data. They’re useful working with small datasets, debugging results, or when running either training or prediction from an API that serves one-off requests.
Spark NLP LightPipelines are Spark ML pipelines converted into a single machine but the multi-threaded task, becoming more than 10x times faster for smaller amounts of data (small is relative, but 50k sentences are roughly a good maximum). To use them, we simply plug in a trained (fitted) pipeline and then annotate a plain text. We don't even need to convert the input text to DataFrame in order to feed it into a pipeline that's accepting DataFrame as an input in the first place. This feature would be quite useful when it comes to getting a prediction for a few lines of text from a trained ML model.

In [0]:
light_model = nlp.LightPipeline(model)

In [0]:
lp_results = light_model.fullAnnotate(sample_text)

**RE Results with LP Function**

Lets create function to get the results in a pandas dataframe.

In [0]:
def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.metadata['sentence'],
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity1'], 
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['entity2'],
          rel.metadata['hypothesis'],
          rel.metadata['nli_prediction'],
          rel.result, 
          rel.metadata['confidence'],
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['sentence', 'entity1_begin','entity1_end','chunk1','entity1','entity2_begin','entity2_end','chunk2', 'entity2','hypothesis', 'nli_prediction', 'relation', 'confidence'])

  return rel_df

In [0]:
rel_df = get_relations_df(lp_results)

print(sample_text, "\n\n")
rel_df

Paracetamol can alleviate headache or sickness. An MRI test can be used to find cancer. 




,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,hypothesis,nli_prediction,relation,confidence
0,0,0,10,Paracetamol,DRUG,38,45,sickness,PROBLEM,Paracetamol improves sickness.,entail,IMPROVE,0.98819494
1,0,0,10,Paracetamol,DRUG,26,33,headache,PROBLEM,Paracetamol improves headache.,entail,IMPROVE,0.9929625
2,1,48,58,An MRI test,TEST,80,85,cancer,PROBLEM,An MRI test reveals cancer.,entail,REVEAL,0.9760039


**As you can see, the results of LP are the same with Spark DF.**

## Visualization of Extracted Relations

We use `RelationExtractionVisualizer` method of `spark-nlp-display` library for visualization fo the extracted relations between the entities.

In [0]:
vis = nlp.viz.RelationExtractionVisualizer()
re_vis = vis.display(lp_results[0], 'relations', show_relations=True, return_html=True)
displayHTML(re_vis)

Paracetamol DRUG can alleviate headache PROBLEM or sickness PROBLEM . An MRI test TEST can be used to find cancer PROBLEM . <polyline fill="none" points="90.0,55.0 92.4795918367347,53.79607986468223 94.95918367346938,52.593128713376224 97.43877551020408,51.39211553009376 99.91836734693878,50.194009298846574 102.39795918367346,48.999779003646445 104.87755102040815,47.810393628505125 107.35714285714288,46.626822157434425 109.83673469387756,45.450033574446024 112.31632653061226,44.280996863551756 114.79591836734693,43.12068100876335 117.2755102040816,41.97005499409259 119.75510204081631,40.83008780355124 122.23469387755101,39.70174842115105 124.71428571428571,38.586005830903794 127.19387755102042,37.48382901682123 129.67346938775512,36.396186962915124 132.15306122448982,35.32404865319723 134.63265306122446,34.26838307167932 137.1122448979592,33.23015920237316 139.59183673469386,32.21034602929051 142.07142857142856,31.209912536443145 144.55102040816325,30.229827707842823 147.03061224489795,29.271060527501298 149.51020408163265,28.334579979430348 151.98979591836735,27.421355047641715 154.46938775510202,26.532354716147186 156.94897959183672,25.668547968958514 159.42857142857144,24.83090379008747 161.90816326530614,24.020391163545803 164.3877551020408,23.237979073345294 166.8673469387755,22.484636503497697 169.34693877551018,21.761332438014776 171.82653061224488,21.069035860908297 174.30612244897958,20.408715756190023 176.78571428571428,19.781341107871725 179.26530612244898,19.187880899965155 181.74489795918367,18.629304116482082 184.22448979591837,18.106579741434267 186.70408163265307,17.620676758833483 189.18367346938774,17.172564152691482 191.66326530612244,16.76321090702004 194.14285714285717,16.393586005830905 196.62244897959184,16.064658433135854 199.1020408163265,15.77739717294665 201.5816326530612,15.53277120927505 204.06122448979593,15.33174952613282 206.5408163265306,15.175301107531727 209.0204081632653,15.064394937483533 211.5,15.0 211.5,15.0 213.97959183673467,14.982762284422305 216.45918367346934,15.012035801409274 218.9387755102041,15.086851566949145 221.4183673469388,15.20624059703015 223.89795918367346,15.369233907640522 226.37755102040813,15.574862514768505 228.8571428571429,15.822157434402333 231.3367346938776,16.11014968253024 233.81632653061223,16.43787027514046 236.2959183673469,16.80435022822123 238.77551020408163,17.208620557760792 241.2551020408163,17.64971227974738 243.734693877551,18.126656410169232 246.2142857142857,18.638483965014576 248.69387755102042,19.184225960271654 251.1734693877551,19.762913411928704 253.65306122448982,20.37357733597396 256.1326530612245,21.01524874839565 258.61224489795916,21.686958665182026 261.09183673469386,22.387738102321304 263.57142857142856,23.116618075801746 266.05102040816325,23.87262960161157 268.53061224489795,24.654803695739016 271.01020408163265,25.462171374172325 273.48979591836735,26.293763652899724 275.969387755102,27.148611547909454 278.44897959183675,28.02574607518976 280.92857142857144,28.92419825072886 283.40816326530614,29.842999090515008 285.88775510204084,30.78117961053642 288.36734693877554,31.737770826781354 290.8469387755102,32.71180375523804 293.32653061224494,33.7023094118947 295.8061224489796,34.70831881273959 298.2857142857143,35.728862973760926 300.765306122449,36.762972910946964 303.2448979591837,37.809679640285935 305.7244897959184,38.868014177766064 308.2040816326531,39.93700753937559 310.68367346938777,41.015690741102766 313.16326530612247,42.103094798935814 315.64285714285717,43.19825072886297 318.1224489795918,44.300189546872474 320.6020408163265,45.40794226895255 323.0816326530612,46.52053991109146 325.5612244897959,47.63701348927743 328.0408163265306,48.75639401949867 330.5204081632653,49.877712517743454 333.0,51.0" stroke="#17C05A" stroke-width="1" /> IMPROVE <polyline fill="none" points="630.0,55.0 633.1224489795917,53.79607986468223 636.2448979591836,52.593128713376224 639.3673469387757,51.39211553009376 642.4897959183673,50.194009298846574

In [0]:
# another example

sample_text2 = "After taking Lipitor, I experienced fatigue and anxiety."

lp_results2 = light_model.fullAnnotate(sample_text2)
re_vis = vis.display(lp_results2[0], 'relations', show_relations=True, return_html=True)
displayHTML(re_vis)

After taking Lipitor DRUG , I experienced fatigue PROBLEM and anxiety PROBLEM . <polyline fill="none" points="192.0,55.0 194.47959183673467,53.79607986468223 196.95918367346937,52.593128713376224 199.43877551020407,51.39211553009376 201.9183673469388,50.194009298846574 204.3979591836735,48.999779003646445 206.87755102040813,47.810393628505125 209.35714285714292,46.626822157434425 211.8367346938776,45.450033574446024 214.31632653061226,44.280996863551756 216.7959183673469,43.12068100876335 219.2755102040816,41.97005499409259 221.75510204081627,40.83008780355124 224.234693877551,39.70174842115105 226.7142857142857,38.586005830903794 229.19387755102042,37.48382901682123 231.6734693877551,36.396186962915124 234.15306122448982,35.32404865319723 236.63265306122446,34.26838307167932 239.1122448979592,33.23015920237316 241.59183673469383,32.21034602929051 244.07142857142853,31.209912536443145 246.55102040816328,30.229827707842823 249.03061224489795,29.271060527501298 251.51020408163268,28.334579979430348 253.98979591836735,27.421355047641715 256.46938775510205,26.532354716147186 258.94897959183675,25.668547968958514 261.42857142857144,24.83090379008747 263.90816326530614,24.020391163545803 266.38775510204084,23.237979073345294 268.86734693877554,22.484636503497697 271.3469387755102,21.761332438014776 273.82653061224494,21.069035860908297 276.3061224489796,20.408715756190023 278.7857142857143,19.781341107871725 281.265306122449,19.187880899965155 283.7448979591837,18.629304116482082 286.2244897959184,18.106579741434267 288.704081632653,17.620676758833483 291.18367346938777,17.172564152691482 293.66326530612247,16.76321090702004 296.14285714285717,16.393586005830905 298.6224489795918,16.064658433135854 301.1020408163265,15.77739717294665 303.5816326530612,15.53277120927505 306.0612244897959,15.33174952613282 308.5408163265306,15.175301107531727 311.0204081632653,15.064394937483533 313.5,15.0 313.5,15.0 315.9795918367347,14.982762284422305 318.45918367346934,15.012035801409274 320.9387755102041,15.086851566949145 323.41836734693874,15.20624059703015 325.89795918367344,15.369233907640522 328.37755102040813,15.574862514768505 330.8571428571429,15.822157434402333 333.3367346938776,16.11014968253024 335.81632653061234,16.43787027514046 338.295918367347,16.80435022822123 340.7755102040816,17.208620557760792 343.2551020408163,17.64971227974738 345.73469387755097,18.126656410169232 348.2142857142858,18.638483965014576 350.6938775510204,19.184225960271654 353.1734693877551,19.762913411928704 355.6530612244899,20.37357733597396 358.13265306122446,21.01524874839565 360.6122448979592,21.686958665182026 363.0918367346938,22.387738102321304 365.57142857142856,23.116618075801746 368.05102040816325,23.87262960161157 370.53061224489795,24.654803695739016 373.0102040816327,25.462171374172325 375.48979591836735,26.293763652899724 377.96938775510205,27.148611547909454 380.44897959183675,28.02574607518976 382.92857142857144,28.92419825072886 385.40816326530614,29.842999090515008 387.88775510204084,30.78117961053642 390.36734693877554,31.737770826781354 392.8469387755102,32.71180375523804 395.32653061224494,33.7023094118947 397.8061224489796,34.70831881273959 400.2857142857143,35.728862973760926 402.7653061224489,36.762972910946964 405.2448979591837,37.809679640285935 407.7244897959183,38.868014177766064 410.204081632653,39.93700753937559 412.6836734693877,41.015690741102766 415.1632653061224,42.103094798935814 417.6428571428571,43.19825072886297 420.1224489795918,44.300189546872474 422.6020408163265,45.40794226895255 425.08163265306126,46.52053991109146 427.5612244897959,47.63701348927743 430.0408163265306,48.75639401949867 432.5204081632653,49.877712517743454 435.0,51.0" stroke="#098CB9" stroke-width="1" /> ADE <polyline fill="none" points="187.0,55.0 190.91836734693877,53.79607986468223 194.8367346938775,52.593128713376224 198.75510204081633,51.39211553009376 202.67346938775512,50.194009298846574 206.59183673469386,48.999779003646445 210.51020408163262,47

## Negative Relationship

**The Implementation Of A Negative Label Feature For Increased Accuracy In The Zero Shot Relation Extraction Model**


`setNegativeRelationships` parameter to the `ZeroShotRelationExtractionModel` annotator, empowering users to exercise more effective control over the model’s predictions for enhanced accuracy. This innovative parameter generates negative examples of relations and subsequently removes them, resulting in improved precision for positive labels.

This advanced feature demonstrates our ongoing commitment to delivering state-of-the-art solutions for healthcare professionals and researchers, facilitating more accurate analysis of complex relationships within clinical text and ultimately contributing to better patient care and outcomes.

In [0]:
re_model = medical.ZeroShotRelationExtractionModel.pretrained("re_zeroshot_biobert", "en", "clinical/models")\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")\
    .setRelationalCategories({
                            "ADE": ["{DRUG} causes {PROBLEM}."],
                            "IMPROVE": ["{DRUG} improves {PROBLEM}.",
                                        "{DRUG} cures {PROBLEM}."],
                            "REVEAL": ["{TEST} reveals {PROBLEM}."]})\
    .setMultiLabel(True)\
    .setNegativeRelationships(["IMPROVE"])


pipeline = nlp.Pipeline() \
    .setStages([documenter,
                sentencer,
                tokenizer,
                words_embedder,
                ner_clinical,
                ner_clinical_converter,
                ner_posology,
                ner_posology_converter,
                chunk_merger,
                pos_tagger,
                dependency_parser,
                re_ner_chunk_filter,
                re_model])

re_zeroshot_biobert download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


In [0]:
# create Spark DF

sample_text = "Paracetamol can alleviate headache or sickness. An MRI test can be used to find cancer."

data = spark.createDataFrame([[sample_text]]).toDF("text")
data.show(truncate=False)

+---------------------------------------------------------------------------------------+
|text                                                                                   |
+---------------------------------------------------------------------------------------+
|Paracetamol can alleviate headache or sickness. An MRI test can be used to find cancer.|
+---------------------------------------------------------------------------------------+



In [0]:
model = pipeline.fit(data)
results = model.transform(data)

In [0]:
# relations output

results.selectExpr("explode(relations) as relation").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|relation                                                                                                                                                                                                                                                                                                                                                  |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
light_model = nlp.LightPipeline(model)

lp_results = light_model.fullAnnotate(sample_text)

In [0]:
rel_df = get_relations_df(lp_results)

print(sample_text, "\n\n")
rel_df

Paracetamol can alleviate headache or sickness. An MRI test can be used to find cancer. 




,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,hypothesis,nli_prediction,relation,confidence
0,1,48,58,An MRI test,TEST,80,85,cancer,PROBLEM,An MRI test reveals cancer.,entail,REVEAL,0.9760039


In [0]:
re_vis = vis.display(lp_results[0], 'relations', show_relations=True, return_html=True)
displayHTML(re_vis)

Paracetamol can alleviate headache or sickness. An MRI test TEST can be used to find cancer PROBLEM . <polyline fill="none" points="571.0,55.0 574.1224489795917,53.79607986468223 577.2448979591836,52.593128713376224 580.3673469387755,51.39211553009376 583.4897959183673,50.194009298846574 586.612244897959,48.999779003646445 589.734693877551,47.810393628505125 592.857142857143,46.626822157434425 595.9795918367348,45.450033574446024 599.1020408163266,44.280996863551756 602.2244897959182,43.12068100876335 605.34693877551,41.97005499409259 608.4693877551019,40.83008780355124 611.5918367346939,39.70174842115105 614.7142857142857,38.586005830903794 617.8367346938776,37.48382901682123 620.9591836734694,36.396186962915124 624.0816326530613,35.32404865319723 627.204081632653,34.26838307167932 630.3265306122448,33.23015920237316 633.4489795918366,32.21034602929051 636.5714285714284,31.209912536443145 639.6938775510205,30.229827707842823 642.8163265306122,29.271060527501298 645.9387755102041,28.334579979430348 649.061224489796,27.421355047641715 652.1836734693877,26.532354716147186 655.3061224489796,25.668547968958514 658.4285714285716,24.83090379008747 661.5510204081633,24.020391163545803 664.6734693877551,23.237979073345294 667.795918367347,22.484636503497697 670.9183673469388,21.761332438014776 674.0408163265306,21.069035860908297 677.1632653061224,20.408715756190023 680.2857142857142,19.781341107871725 683.4081632653061,19.187880899965155 686.530612244898,18.629304116482082 689.6530612244899,18.106579741434267 692.7755102040816,17.620676758833483 695.8979591836735,17.172564152691482 699.0204081632653,16.76321090702004 702.1428571428572,16.393586005830905 705.2653061224489,16.064658433135854 708.3877551020407,15.77739717294665 711.5102040816327,15.53277120927505 714.6326530612245,15.33174952613282 717.7551020408163,15.175301107531727 720.8775510204082,15.064394937483533 724.0,15.0 724.0,15.0 727.1224489795918,14.982762284422305 730.2448979591835,15.012035801409274 733.3673469387757,15.086851566949145 736.4897959183675,15.20624059703015 739.6122448979592,15.369233907640522 742.7346938775509,15.574862514768505 745.857142857143,15.822157434402333 748.9795918367347,16.11014968253024 752.1020408163267,16.43787027514046 755.2244897959182,16.80435022822123 758.3469387755101,17.208620557760792 761.469387755102,17.64971227974738 764.5918367346937,18.126656410169232 767.7142857142857,18.638483965014576 770.8367346938777,19.184225960271654 773.9591836734695,19.762913411928704 777.0816326530613,20.37357733597396 780.204081632653,21.01524874839565 783.3265306122448,21.686958665182026 786.4489795918366,22.387738102321304 789.5714285714286,23.116618075801746 792.6938775510205,23.87262960161157 795.8163265306122,24.654803695739016 798.9387755102041,25.462171374172325 802.0612244897959,26.293763652899724 805.1836734693877,27.148611547909454 808.3061224489795,28.02574607518976 811.4285714285714,28.92419825072886 814.5510204081633,29.842999090515008 817.6734693877551,30.78117961053642 820.795918367347,31.737770826781354 823.9183673469387,32.71180375523804 827.0408163265306,33.7023094118947 830.1632653061224,34.70831881273959 833.2857142857143,35.728862973760926 836.408163265306,36.762972910946964 839.530612244898,37.809679640285935 842.6530612244898,38.868014177766064 845.7755102040817,39.93700753937559 848.8979591836735,41.015690741102766 852.0204081632653,42.103094798935814 855.1428571428571,43.19825072886297 858.2653061224489,44.300189546872474 861.3877551020407,45.40794226895255 864.5102040816325,46.52053991109146 867.6326530612245,47.63701348927743 870.7551020408164,48.75639401949867 873.8775510204081,49.877712517743454 877.0,51.0" stroke="#670F97" stroke-width="1" /> REVEAL

- Without Setting setNegativeRelationships:

|sentence|entity1_begin|entity1_end|chunk1|entity1|entity2_begin|entity2_end|chunk2|entity2|hypothesis|nli_prediction|relation|confidence|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|0|0|10|Paracetamol|DRUG|38|45|sickness|PROBLEM|Paracetamol improves sickness.|entail|IMPROVE|0.98819494|
|0|0|10|Paracetamol|DRUG|26|33|headache|PROBLEM|Paracetamol improves headache.|entail|IMPROVE|0.9929625|
|1|48|58|An MRI test|TEST|80|85|cancer|PROBLEM|An MRI test reveals cancer.|entail|REVEAL|0.9760039|




  - After Setting setNegativeRelationships:

|sentence|entity1_begin|entity1_end|chunk1|entity1|entity2_begin|entity2_end|chunk2|entity2|hypothesis|nli_prediction|relation|confidence|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|1|48|58|An MRI test|TEST|80|85|cancer|PROBLEM|An MRI test reveals cancer.|entail|REVEAL|0.9760039|

# Save the Model and Load from Disc

After creating the Hypothesis, we can save the model and load it from disc without setting releatin categories.

In [0]:
# save model

re_model.write().overwrite().save("dbfs:/databricks/driver/zeroshot_re_model")

In [0]:
# load from disc and create a new pipeline

re_model2 = medical.ZeroShotRelationExtractionModel.load("dbfs:/databricks/driver/zeroshot_re_model")\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")


pipeline2 = nlp.Pipeline() \
    .setStages([documenter,  
                sentencer,
                tokenizer, 
                words_embedder, 
                ner_clinical, 
                ner_clinical_converter,
                ner_posology, 
                ner_posology_converter,
                chunk_merger,
                pos_tagger, 
                dependency_parser, 
                re_ner_chunk_filter, 
                re_model2])
    
model2 = pipeline2.fit(data)
results2 = model2.transform(data)

In [0]:
re_model2.getClasses()

['IMPROVE', 'ADE', 'REVEAL']

In [0]:
# results of the new pipeline

results2.selectExpr("explode(relations) as relation").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|relation                                                                                                                                                                                                                                                                                                                                                  |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

As you can see above, we got the same results by loading our saved ZSL model from disc although we didn't set any relation categories.